In [ ]:
import pandas as pd
import torch
from transformers import pipeline


## Test HuggingFace call


In [ ]:
# sentiment_test = pipeline(
#    "sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

In [ ]:
# response = sentiment_test(
#    "I think my Spotify AI dj is learning that I'm autistic and only like listening to the songs I already have on my playlist"
# )

In [ ]:
# response

## Sentiment Analysis on collected tweets


In [ ]:
tweets = pd.read_csv("data/spotify_ai_dj_tweets_clean.csv")
tweets.head()

In [ ]:
# Run sentiment analysis on the 'text_clean' column using the classifier and device selection above


# Only consider cuda or cpu, ignore mps


device = "cuda" if torch.cuda.is_available() else "cpu"

if device == "cuda":
    classifier = pipeline(
        "sentiment-analysis",
        model="cardiffnlp/twitter-roberta-base-sentiment-latest",
        device=0,
    )
else:
    classifier = pipeline(
        "sentiment-analysis",
        model="cardiffnlp/twitter-roberta-base-sentiment-latest",
        device=-1,
    )


# Get both sentiment and confidence (score) for each tweet
def get_sentiment_and_confidence(text):
    result = classifier(text)[0]
    return pd.Series({"sentiment": result["label"], "confidence": result["score"]})


tweets[["sentiment", "confidence"]] = tweets["text_clean"].apply(
    get_sentiment_and_confidence
)
tweets.head()
